In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/yono

/content/drive/MyDrive/yono


In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 88 kB/s 


In [ ]:
!pip install pytorchcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 532 kB 5.3 MB/s 


In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from __future__ import absolute_import, division, print_function
import os, sys, json
current_path = os.path.abspath('.')
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import argparse
import collections,math
import time,datetime,pytz
import numpy as np
import pandas as pd
import pickle
import unittest

from scipy.fftpack import dct, idct
from scipy import stats
from scipy import interpolate, io
from scipy.io import loadmat
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
import operator
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torchvision import datasets, transforms
from tqdm import tqdm
from torchinfo import summary
import faiss

from src.data_loader import *
from src.utils import *

from src.models.simple_cnn import *
from src.models.resnet_models import *
from src.models.dscnn import *
from src.train_joint import *

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

############################################
########## Plot Style Declaration ##########
# Set the style globally
# Alternatives include bmh, fivethirtyeight, ggplot,
# dark_background, seaborn-deep, etc
# plt.style.use('ggplot')
plt.style.use('seaborn-white')

plt.rcParams['font.family'] = 'times new roman'
# plt.rcParams['font.serif'] = 'Ubuntu'
# plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 15
plt.rcParams['axes.labelsize'] = 15
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 15
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 15
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

##################################
########## End of Setup ##########

In [ ]:
class ArgClass():
    def __init__(self):
        super(ArgClass, self).__init__()

args = ArgClass()
args.dataset="mnist"
args.model_arch = "cnn"
args.batch_size=128
args.test_batch_size=1000
args.percent=[0.8, 0.92, 0.991, 0.93]
args.alpha=5e-4
args.rho=1e-2
args.l1=False
args.l2=False
args.num_pre_epochs=3
args.num_epochs=10
args.num_re_epochs=3
args.lr=1e-3
args.adam_epsilon=1e-8
args.no_cuda=True
args.seed=1
args.save_model=False

args.optimizer_name = 'adam'
args.lr_mode = 'multistep'
args.lr_decay = 0.1
args.lr_decay_epoch = '20,40'
args.target_lr = 1e-8
args.warmup_epochs = 0
args.warmup_lr = 1e-8
args.warmup_mode = 'linear'
args.momentum = 0.9
args.wd = 0.0001
args.gamma_wd_mult = 1.0
args.beta_wd_mult = 1.0
args.bias_wd_mult = 1.0
args.grad_clip = None
args.label_smoothing = False


args.test_fold_l = '[10]'
args.use_one_task = 'false'
args.exp_setup = ''
args.subject_idx = None
args.pretrained=False
args.best_acc = 200.0
args.mixup = False
args.mixup_alpha = 1.0
args.mixup_epoch_tail = 10
args.session = 1

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [ ]:

# mnist = VWNet(args,
#             in_channels=1,
#             in_size=(28,28),
#             channels=[8,16,32],
#             fcDims=[128,256],
#             kernels=[(3,3)],
#             num_classes=10,
#             pooling_type='max')

# 90.05
args.config = DATASET_CONFIGS['cifar10']
cifar10 = PTCVDSCNN(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
                             num_classes=args.config["classes"],
                             init_block_kernel=(3, 3),
                             pretrained=False)


# # 94.48
# args.config = DATASET_CONFIGS['svhn']
# svhn = PTCVDSCNN(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
#                              num_classes=args.config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=False)

# # 97.50
# args.config = DATASET_CONFIGS['gtsrb-vw']
# gtsrb = PTCVDSCNN(model_name='MicroNet-AD-I-ch168', in_channels=args.config["in_channels"],
#                              num_classes=args.config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=False)

# args.config = DATASET_CONFIGS['gscv2-vw-c35']
# gsc = PTCVDSCNN(model_name='MicroNet-M', in_channels=args.config["in_channels"],
#                              num_classes=args.config["classes"],
#                              init_block_kernel=(10, 4),
#                              pretrained=False)



In [ ]:
# 2 Image
# config = DATASET_CONFIGS['fashion-mnist']
# fmnist = PTCVDSCNN(model_name='DS-CNN-M', in_channels=config["in_channels"],
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)

# config = DATASET_CONFIGS['stl10']
# stl10 = PTCVDSCNN(model_name='DS-CNN-M-s33', in_channels=config["in_channels"],
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)

# # 2 Audio
# config = DATASET_CONFIGS['emotion-5-frames25']
# emotion = AudioCNN(channels=[32,32,64,64],
#                              in_size=(config["features"],config["seq"] ),
#                              num_classes=config['classes'],
#                              pooling_type="avg")

# config = DATASET_CONFIGS['urbansound8k-LMCST-long-randfold']
# urbansound = PTCVDSCNN(model_name='DS-CNN-M-s33', in_channels=1,
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)


# # 3 HAR
# config = DATASET_CONFIGS['hhar-noaug']
# hhar = PTCVDSCNN(model_name='MicroNet-AD-I-ch176-s41', in_channels=1,
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)


# config = DATASET_CONFIGS['pamap2']
# pamap2 = PTCVDSCNN(model_name='MicroNet-AD-I-ch168-s21', in_channels=1,
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)

# # 97.65
# config = DATASET_CONFIGS['skoda']
# skoda = PTCVDSCNN(model_name='MicroNet-AD-I-ch192-s22', in_channels=1,
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)


# # 3 sEMG
# config = DATASET_CONFIGS['ninapro-db2-c10']
# ninadb2 = AudioCNN(channels=[16,16,32,32],
#                              in_size=(config["features"],config["seq"] ),
#                              num_classes=config['classes'],
#                              pooling_type="avg")

# config = DATASET_CONFIGS['ninapro-db3-c10']
# ninadb3 = AudioCNN(channels=[16,16,32,32],
#                              in_size=(config["features"],config["seq"] ),
#                              num_classes=config['classes'],
#                              pooling_type="avg")


# config = DATASET_CONFIGS['ninapro-db6-c7-seq80']
# ninadb6 = PTCVDSCNN(model_name='MicroNet-AD-K', in_channels=1,
#                              num_classes=config["classes"],
#                              init_block_kernel=(3, 3),
#                              pretrained=args.pretrained)


In [ ]:
def getModelAndIndexAndCodebookSizes(model, cr3x3=9, cr1x1=4):
    conv3x3_cnt = 0
    conv1x1_cnt = 0
    tot_param_cnt = 0
    conv3x3_codebook_cnt = 0
    conv1x1_codebook_cnt = 0
    fc_codebook_cnt = 0
    float_cnt = 0
    for i, param in enumerate(model.parameters()):
        size_tot = param.numel()
        tot_param_cnt += size_tot
        if len(param.size()) == 4: # consider only conv weights
            if i == 0: # CASE for init_block (init block is in 32bit float)
                float_cnt += size_tot
            else: # CASE for middle blocks
                if param.size()[2] == 3 and param.size()[3] == 3:
                    conv3x3_cnt += size_tot
                    conv3x3_codebook_cnt += 1
                else:
                    conv1x1_cnt += size_tot
                    conv1x1_codebook_cnt += 1
        elif len(param.size()) == 2: # CASE for last layer / linear layers
            conv1x1_cnt += size_tot
            fc_codebook_cnt += 1
        elif len(param.size()) == 1: # CASE for bias (32 bit float)
            float_cnt += size_tot
        else:
            print('error: this should not be printed, i.e., (param.size() == 3 or > 4 ) ')
    model_size = tot_param_cnt * 4 / 1024 / 1024 # (in MB)
    index_size = conv3x3_cnt / cr3x3 / 1024 / 1024 + conv1x1_cnt / cr1x1 / 1024 / 1024 # (in MB)
    codebook_size = conv3x3_codebook_cnt * getCodebookSizeConv(cr3x3) +\
                    conv1x1_codebook_cnt * getCodebookSizeConv(cr1x1) +\
                    fc_codebook_cnt * 1024*cr1x1*2/1024/1024    # (in MB)
    others_size = float_cnt * 4 / 1024 / 1024 # (in MB)
    
    print(f'model_size: {model_size:.2f} MB, index_size: {index_size:.2f} MB, codebook_size: {codebook_size:.2f} MB, Init_block and Bias size: {others_size:.2f} MB')
    print(f'(Index+Codebook+Others)/Model: {(index_size+codebook_size+others_size)/model_size*100:.2f}%, Comp Rate: {model_size/(index_size+codebook_size+others_size):.2f}x')
    return model_size, index_size, codebook_size, others_size

def getCodebookSizeConv(comp_rate=9):
    numel = 256*comp_rate
    codebook_size = (numel)*2/1024/1024 # (in MB)
    return codebook_size

def printiPQCompRate(models, cr3x3=9, cr1x1=4):
    model_size_l = []
    index_size_l = []
    codebook_size_l = []
    others_size_l = []
    
    for i, model in enumerate(models):
        model_size, index_size, codebook_size, others_size = getModelAndIndexAndCodebookSizes(model, cr3x3, cr1x1)
        model_size_l.append(model_size)
        index_size_l.append(index_size)
        codebook_size_l.append(codebook_size)
        others_size_l.append(others_size)
    
    tot_model_size = np.sum(model_size_l)
    tot_index_size = np.sum(index_size_l)
    tot_codebook_size = np.sum(codebook_size_l)
    tot_others_size = np.sum(others_size_l)
    
    print(f'\nTOTAL model_size: {tot_model_size:.2f} MB, \
  TOTAL index_size: {tot_index_size:.2f} MB, \
  TOTAL Codebook Size: {tot_codebook_size:.2f} MB, \
  TOTAL Others Size: {tot_others_size:.2f} MB')
    print(f'TOTAL Index/Model Ratio: {tot_index_size/tot_model_size*100:.2f}%, \
    Codebook/Model Ratio: {tot_codebook_size/tot_model_size*100:.2f}%, \
    Others/Model Ratio: {tot_others_size/tot_model_size*100:.2f}%')
    print(f'\nTOTAL Comp Rate: {tot_model_size/(tot_index_size+tot_codebook_size+tot_others_size):.2f}x\n\n')


In [ ]:
# print('Sound')
# test(args, cnn_emotion.to(device), device, test_loader_emotion5)
# test(args, dscnnL_urbansound.to(device), device, test_loader_urbansound)
# test(args, microL_gscv2.to(device), device, test_loader_gscv2)

# print('HAR')
# test(args, microL_hhar.to(device), device, test_loader_hhar)
# test(args, microL_pamap2.to(device), device, test_loader_pamap2)
# test(args, microL_skoda.to(device), device, test_loader_skoda)

# print('sEMG')
# test(args, cnn_ninadb2.to(device), device, test_loader_ninadb2)
# test(args, cnn_ninadb3.to(device), device, test_loader_ninadb3)
# test(args, microL_ninadb6.to(device), device, test_loader_ninadb6)

def getModelAndIndexSizes(model, cr3x3=9, cr1x1=4, n_layers_tune=2, reverse_tune=False):
    conv3x3_cnt = 0
    conv1x1_cnt = 0
    tot_param_cnt = 0
    conv3x3_codebook_cnt = 0
    conv1x1_codebook_cnt = 0
    fc_codebook_cnt = 0
    fc_cnt = 0
    float_cnt = 0
    
    conv_param_l = []
    for i, param in enumerate(model.parameters()):
        size_tot = param.numel()
        tot_param_cnt += size_tot
        if len(param.size()) == 4: # consider only conv weights
            if i == 0: # CASE for init_block (init block is in 32bit float)
                # float_cnt += size_tot
                fc_cnt += size_tot
            else: # CASE for middle blocks
                conv_param_l.append(size_tot)
                if param.size()[2] == 1 and param.size()[3] == 1:
                    conv1x1_cnt += size_tot
                else:
                    conv3x3_cnt += size_tot
        elif len(param.size()) == 2: # CASE for last layer / linear layers
            size_l = param.size()
            last_layer_flag = False
            for temp in size_l:
                if temp < 50:
                    last_layer_flag = True
            if last_layer_flag == True:
                fc_cnt += size_tot
            else:
                conv1x1_cnt += size_tot
            # fc_codebook_cnt += 1
        elif len(param.size()) == 1: # CASE for bias (32 bit float)
            float_cnt += size_tot
        else:
            print('error: this should not be printed, i.e., (param.size() == 3 or > 4 ) ')
    
    # if n_layers_tune > 2: we add fc_cnt and remove conv1x1_cnt or conv3x3_cnt
    if n_layers_tune > 2:
        for i in range(n_layers_tune - 2):
            if reverse_tune is False:
                if i % 2 == 0: # decrease conv1x1
                    conv1x1_cnt -= conv_param_l[-1-i]
                    fc_cnt += conv_param_l[-1-i]
                else: # decrease conv3x3
                    conv3x3_cnt -= conv_param_l[-1-i]
                    fc_cnt += conv_param_l[-1-i]
            else:
                if i % 2 == 0: # decrease conv3x3
                    conv3x3_cnt -= conv_param_l[i]
                    fc_cnt += conv_param_l[i]
                else: # decrease conv1x1
                    conv1x1_cnt -= conv_param_l[i]
                    fc_cnt += conv_param_l[i]
            
    model_size = tot_param_cnt * 4 / 1024 / 1024 # (in MB)
    index_size = conv3x3_cnt / cr3x3 / 1024 / 1024 + conv1x1_cnt / cr1x1 / 1024 / 1024 # (in MB)
    codebook_size = fc_codebook_cnt * 1024*cr1x1*2/1024/1024    # (in MB)
    others_size = float_cnt * 4 / 1024 / 1024 + fc_cnt * 2 /1024/1024 # (in MB)
    
    
    print(f'model_size: {model_size:.2f} MB, index_size: {index_size:.2f} MB, codebook_size: {codebook_size:.2f} MB, Init_block and Bias size: {others_size:.2f} MB')
    print(f'(Index+Codebook+Others)/Model: {(index_size+codebook_size+others_size)/model_size*100:.2f}%, Comp Rate: {model_size/(index_size+codebook_size+others_size):.2f}x')
    return model_size, index_size, codebook_size, others_size

def getCodebookSize(comp_rate3x3=9, comp_rate1x1=4):
    conv3x3_numel = 256*comp_rate3x3*2
    conv1x1_numel = 256*comp_rate1x1*2
    codebook_size = (conv3x3_numel + conv1x1_numel)*2/1024/1024 # (in MB)
    return codebook_size

def printOverallCompRate(models, cr3x3=9, cr1x1=4, n_layers_tune=2, reverse_tune=False):
    model_size_l = []
    index_size_l = []
    codebook_size_l = []
    others_size_l = []
    shared_codebook_size = getCodebookSize(cr3x3, cr1x1)
    
    for i, model in enumerate(models):
        model_size, index_size, codebook_size, others_size = getModelAndIndexSizes(model, cr3x3, cr1x1, n_layers_tune, reverse_tune)
        model_size_l.append(model_size)
        index_size_l.append(index_size)
        codebook_size_l.append(codebook_size)
        others_size_l.append(others_size)
    
    tot_model_size = np.sum(model_size_l)
    tot_index_size = np.sum(index_size_l)
    tot_codebook_size = np.sum(codebook_size_l)
    tot_others_size = np.sum(others_size_l)
    tot_codebook_size += shared_codebook_size
    
    print(f'\nTOTAL model_size: {tot_model_size:.2f} MB, \
  TOTAL index_size: {tot_index_size:.2f} MB, \
  TOTAL Codebook Size: {tot_codebook_size:.2f} MB, \
  TOTAL Others Size: {tot_others_size:.2f} MB')
    print(f'TOTAL Index/Model Ratio: {tot_index_size/tot_model_size*100:.2f}%, \
    Codebook/Model Ratio: {tot_codebook_size/tot_model_size*100:.2f}%, \
    Others/Model Ratio: {tot_others_size/tot_model_size*100:.2f}%')
    print(f'\nTOTAL Comp Rate: {tot_model_size/(tot_index_size+tot_codebook_size+tot_others_size):.2f}x')
    print(f'\nTOTAL YONO Size: {tot_index_size+tot_codebook_size+tot_others_size:.2f} MB\n\n')
    
def printOverallCompRateList(models, n_layers_tune_l, cr3x3=9, cr1x1=4, reverse_tune=False):
    model_size_l = []
    index_size_l = []
    codebook_size_l = []
    others_size_l = []
    shared_codebook_size = getCodebookSize(cr3x3, cr1x1)
    
    for i, (model, n_layers_tune) in enumerate(zip(models,n_layers_tune_l)):
        model_size, index_size, codebook_size, others_size = getModelAndIndexSizes(model, cr3x3, cr1x1, n_layers_tune, reverse_tune)
        model_size_l.append(model_size)
        index_size_l.append(index_size)
        codebook_size_l.append(codebook_size)
        others_size_l.append(others_size)
    
    tot_model_size = np.sum(model_size_l)
    tot_index_size = np.sum(index_size_l)
    tot_codebook_size = np.sum(codebook_size_l)
    tot_others_size = np.sum(others_size_l)
    tot_codebook_size += shared_codebook_size
    
    print(f'\nTOTAL model_size: {tot_model_size:.2f} MB, \
  TOTAL index_size: {tot_index_size:.2f} MB, \
  TOTAL Codebook Size: {tot_codebook_size:.2f} MB, \
  TOTAL Others Size: {tot_others_size:.2f} MB')
    print(f'TOTAL Index/Model Ratio: {tot_index_size/tot_model_size*100:.2f}%, \
    Codebook/Model Ratio: {tot_codebook_size/tot_model_size*100:.2f}%, \
    Others/Model Ratio: {tot_others_size/tot_model_size*100:.2f}%')
    print(f'\nTOTAL Comp Rate: {tot_model_size/(tot_index_size+tot_codebook_size+tot_others_size):.2f}x')
    print(f'\nTOTAL YONO Size: {tot_index_size+tot_codebook_size+tot_others_size:.2f} MB\n\n')
    
def printOverallCompRate4SingleModel(models, cr3x3=9, cr1x1=4, n_layers_tune=2, reverse_tune=False):
    model_size_l = []
    index_size_l = []
    codebook_size_l = []
    others_size_l = []
    shared_codebook_size = getCodebookSize(cr3x3, cr1x1)
    
    for i, model in enumerate(models):
        model_size, index_size, codebook_size, others_size = getModelAndIndexSizes(model, cr3x3, cr1x1, n_layers_tune, reverse_tune)
        model_size_l.append(model_size)
        index_size_l.append(index_size)
        codebook_size_l.append(codebook_size)
        others_size_l.append(others_size)
    
    tot_model_size = np.sum(model_size_l)
    tot_index_size = np.sum(index_size_l)
    tot_codebook_size = np.sum(codebook_size_l)
    tot_others_size = np.sum(others_size_l)
    tot_codebook_size += shared_codebook_size*len(models)
    print('shared_codebook_size: ', shared_codebook_size)
    
    print(f'\nTOTAL model_size: {tot_model_size:.2f} MB, \
  TOTAL index_size: {tot_index_size:.2f} MB, \
  TOTAL Codebook Size: {tot_codebook_size:.2f} MB, \
  TOTAL Others Size: {tot_others_size:.2f} MB')
    print(f'TOTAL Index/Model Ratio: {tot_index_size/tot_model_size*100:.2f}%, \
    Codebook/Model Ratio: {tot_codebook_size/tot_model_size*100:.2f}%, \
    Others/Model Ratio: {tot_others_size/tot_model_size*100:.2f}%')
    print(f'\nTOTAL Comp Rate: {tot_model_size/(tot_index_size+tot_codebook_size+tot_others_size):.2f}x')
    print(f'\nTOTAL YONO Size: {tot_index_size+tot_codebook_size+tot_others_size:.2f} MB\n\n')

# USED IN PAPER

## Section 4.2 (2 modalities)

In [ ]:
printOverallCompRate4SingleModel([
                                  # mnist,
       cifar10
      #  svhn,
      #  gtsrb,
      #  gsc
       ],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x
shared_codebook_size:  0.0166015625

TOTAL model_size: 0.76 MB,   TOTAL index_size: 0.02 MB,   TOTAL Codebook Size: 0.02 MB,   TOTAL Others Size: 0.03 MB
TOTAL Index/Model Ratio: 2.94%,     Codebook/Model Ratio: 2.19%,     Others/Model Ratio: 3.71%

TOTAL Comp Rate: 11.31x

TOTAL YONO Size: 0.07 MB




In [ ]:
# MNIST, CIFAR10, SVHN, GTSRB, GSC
# 2x, 3x, 3x, 3x, 3x
printOverallCompRate([
                      # mnist,
       cifar10
      #  svhn,
      #  gtsrb,
      #  gsc
       ],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

# printOverallCompRateList([mnist,
#        cifar10,
#        svhn,
#        gtsrb,
#        gsc],
#         [2,3,3,3,3],
#         cr3x3=9, cr1x1=8, reverse_tune=True)

model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x

TOTAL model_size: 0.76 MB,   TOTAL index_size: 0.02 MB,   TOTAL Codebook Size: 0.02 MB,   TOTAL Others Size: 0.03 MB
TOTAL Index/Model Ratio: 2.94%,     Codebook/Model Ratio: 2.19%,     Others/Model Ratio: 3.71%

TOTAL Comp Rate: 11.31x

TOTAL YONO Size: 0.07 MB




## Section 4.3 (4 modalities)

In [ ]:
3.84 - 2.97

0.8699999999999997

In [ ]:
3.76-2.91

0.8499999999999996

In [ ]:
printOverallCompRate4SingleModel([mnist,
       cifar10,
       svhn,
       gtsrb,
       gsc,
        hhar,
        ninadb2],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

model_size: 0.17 MB, index_size: 0.00 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.01 MB
(Index+Codebook+Others)/Model: 10.94%, Comp Rate: 9.14x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x
model_size: 0.77 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 7.38%, Comp Rate: 13.56x
model_size: 0.45 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.04 MB
(Index+Codebook+Others)/Model: 10.61%, Comp Rate: 9.42x
model_size: 0.75 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 5.81%, Comp Rate: 17.20x
model_size: 0.10 MB, index_size: 0.00 MB, codebook_size: 0.00 MB

In [ ]:
# MNIST, CIFAR10, SVHN, GTSRB, GSC, HHAR, Ninadb2
# 2x, 3x, 3x, 3x, 3x, 3x, 2x
printOverallCompRate([mnist,
       cifar10,
       svhn,
       gtsrb,
       gsc,
        hhar,
        ninadb2],
        cr3x3=9,cr1x1=8,n_layers_tune=2, reverse_tune=True)

printOverallCompRateList([mnist,
       cifar10,
       svhn,
       gtsrb,
       gsc,
        hhar,
        ninadb2],
        [2,3,3,3,3,3,2],
        cr3x3=9,cr1x1=8,reverse_tune=True)

model_size: 0.17 MB, index_size: 0.00 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.01 MB
(Index+Codebook+Others)/Model: 10.94%, Comp Rate: 9.14x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 6.65%, Comp Rate: 15.04x
model_size: 0.77 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.03 MB
(Index+Codebook+Others)/Model: 7.38%, Comp Rate: 13.56x
model_size: 0.45 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.04 MB
(Index+Codebook+Others)/Model: 10.61%, Comp Rate: 9.42x
model_size: 0.75 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 5.81%, Comp Rate: 17.20x
model_size: 0.10 MB, index_size: 0.00 MB, codebook_size: 0.00 MB

## Section 4.4 (Including Unseen)

In [ ]:
# MNIST, CIFAR10, SVHN, GTSRB, GSC, HHAR, Ninadb2
# 2x, 3x, 3x, 3x, 3x, 3x, 2x

# FashionMNIST, STL-10,// Emotion-5, urbansound,// PAMAP2, Skoda // NinaproDB3, NinaproDB6
# 3x, 3x,// 2x, 3x,// 2x, 3x,// 2x 3x
printOverallCompRate([fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

printOverallCompRateList([fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        [3,3,2,3,2,3,2,3],
        cr3x3=9, cr1x1=8, reverse_tune=True)

model_size: 0.50 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.58%, Comp Rate: 15.20x
model_size: 0.51 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 7.58%, Comp Rate: 13.19x
model_size: 0.29 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 9.76%, Comp Rate: 10.25x
model_size: 0.50 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.58%, Comp Rate: 15.20x
model_size: 0.75 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.07%, Comp Rate: 16.48x
model_size: 0.70 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 5.92%, Comp Rate: 16.89x
model_size: 0.10 MB, index_size: 0.00 MB, codebook_size: 0.00 MB

In [ ]:
# MNIST, CIFAR10, SVHN, GTSRB, GSC, HHAR, Ninadb2
# 2x, 3x, 3x, 3x, 3x, 3x, 2x

# FashionMNIST, STL-10,// Emotion-5, urbansound,// PAMAP2, Skoda // NinaproDB3, NinaproDB6
# 3x, 3x,// 2x, 3x,// 2x, 3x,// 2x 3x
printOverallCompRate([fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

printOverallCompRateList([fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        [3,3,3,3,2,3,2,3],
        cr3x3=9, cr1x1=8, reverse_tune=True)

model_size: 0.50 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.58%, Comp Rate: 15.20x
model_size: 0.51 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 7.58%, Comp Rate: 13.19x
model_size: 0.29 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 9.76%, Comp Rate: 10.25x
model_size: 0.50 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.58%, Comp Rate: 15.20x
model_size: 0.75 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.07%, Comp Rate: 16.48x
model_size: 0.70 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 5.92%, Comp Rate: 16.89x
model_size: 0.10 MB, index_size: 0.00 MB, codebook_size: 0.00 MB

In [ ]:
12.37/8.08

1.5309405940594059

In [ ]:
12.11/8.08

1.4987623762376237

In [ ]:
# MNIST, CIFAR10, SVHN, GTSRB, GSC, HHAR, Ninadb2
# 2x, 3x, 3x, 3x, 3x, 3x, 2x

# FashionMNIST, STL-10,// Emotion-5, urbansound,// PAMAP2, Skoda // NinaproDB3, NinaproDB6
# 3x, 3x,// 2x, 3x,// 2x, 3x,// 2x 3x
printOverallCompRate([mnist,
       cifar10,
       svhn,
       gtsrb,
       gsc,
        hhar,
        ninadb2, fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        cr3x3=9, cr1x1=8, n_layers_tune=2, reverse_tune=True)

printOverallCompRateList([mnist,
       cifar10,
       svhn,
       gtsrb,
       gsc,
        hhar,
        ninadb2, fmnist,
       stl10,
       emotion,
       urbansound,
       pamap2,
        skoda,
        ninadb3,
        ninadb6],
        [2,3,3,3,3,3,2, 
         3,3,2,3,2,3,2,3],
        cr3x3=9, cr1x1=8, reverse_tune=True)

model_size: 0.17 MB, index_size: 0.00 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.01 MB
(Index+Codebook+Others)/Model: 11.01%, Comp Rate: 9.08x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.04 MB
(Index+Codebook+Others)/Model: 7.79%, Comp Rate: 12.83x
model_size: 0.76 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.04 MB
(Index+Codebook+Others)/Model: 7.79%, Comp Rate: 12.83x
model_size: 0.77 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.04 MB
(Index+Codebook+Others)/Model: 7.75%, Comp Rate: 12.90x
model_size: 0.45 MB, index_size: 0.01 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.05 MB
(Index+Codebook+Others)/Model: 12.98%, Comp Rate: 7.70x
model_size: 0.75 MB, index_size: 0.02 MB, codebook_size: 0.00 MB, Init_block and Bias size: 0.02 MB
(Index+Codebook+Others)/Model: 6.22%, Comp Rate: 16.08x
model_size: 0.10 MB, index_size: 0.00 MB, codebook_size: 0.00 MB